In [1]:
sc.version

'3.3.2'

In [3]:
!wget https://storage.googleapis.com/bd522_2021/data/pg4300.txt

--2024-02-25 02:57:55--  https://storage.googleapis.com/bd522_2021/data/pg4300.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 142.251.175.207, 74.125.24.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572441 (1.5M) [text/plain]
Saving to: ‘pg4300.txt’

pg4300.txt          100%[===================>]   1.50M  --.-KB/s    in 0.01s   

2024-02-25 02:57:55 (151 MB/s) - ‘pg4300.txt’ saved [1572441/1572441]



In [9]:
!hadoop fs -put pg4300.txt /user/dataproc/pg4300.txt

put: `/user/dataproc/pg4300.txt': File exists


In [12]:
!hadoop fs -ls /user/dataproc


Found 1 items
-rw-r--r--   2 root hadoop    1572441 2024-02-25 03:03 /user/dataproc/pg4300.txt


In [16]:
lines = sc.textFile("/user/dataproc/pg4300.txt")
words = lines.flatMap(lambda s: s.split(" "))
word_tuples = words.map(lambda t: (t, 1))
word_count = word_tuples.reduceByKey(lambda x,y: x+y)
word_count.take(10)

[('Ulysses', 4),
 ('', 9901),
 ('Author:', 1),
 ('Date:', 1),
 ('1,', 3),
 ('Last', 20),
 ('January', 4),
 ('29,', 3),
 ('2016', 1),
 ('Language:', 1)]

In [2]:
# Data for Workshop 4
!wget https://storage.googleapis.com/bd522_2021/data/sensor_12.csv

--2024-03-03 02:39:39--  https://storage.googleapis.com/bd522_2021/data/sensor_12.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62711541 (60M) [text/csv]
Saving to: ‘sensor_12.csv’

sensor_12.csv       100%[===================>]  59.81M   165MB/s    in 0.4s    

2024-03-03 02:39:40 (165 MB/s) - ‘sensor_12.csv’ saved [62711541/62711541]



In [3]:
!hadoop fs -put sensor_12.csv /user/dataproc/sensor_12.csv

In [4]:
!hadoop fs -ls /user/dataproc

Found 1 items
-rw-r--r--   2 root hadoop   62711541 2024-03-03 02:39 /user/dataproc/sensor_12.csv


In [5]:
#Read CSV file to Spark Data Frame
df = spark.read.csv("/user/dataproc/sensor_12.csv",
                   sep=";",
                   inferSchema=True,
                   header=True)

In [6]:
df.count()

1000000

In [7]:
# Print structure or column of Dataframe
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- sensor_value1: double (nullable = true)
 |-- station_id: integer (nullable = true)
 |-- sensor_value2: double (nullable = true)



In [25]:
df.show(10)

+-------------------+------------------+----------+------------------+
|               date|     sensor_value1|station_id|     sensor_value2|
+-------------------+------------------+----------+------------------+
|2018-01-01 00:00:00| 6.232350314796101|         1|  9.29195686230464|
|2018-01-01 00:01:00| 8.681493091199476|         1|12.907931284372664|
|2018-01-01 00:02:00|  5.55753392099947|         1|  8.49261966162583|
|2018-01-01 00:03:00|0.6644226937927744|         1|0.9566552990323458|
|2018-01-01 00:04:00|2.1559831028440204|         1| 3.329389247000051|
|2018-01-01 00:05:00| 4.772505630038132|         1|   7.2075621356472|
|2018-01-01 00:06:00|1.8112114353164608|         1|2.6762864414631387|
|2018-01-01 00:07:00| 9.682279044658689|         1| 14.45976515409092|
|2018-01-01 00:08:00| 5.655454912021871|         1| 8.410295455682496|
|2018-01-01 00:09:00|  9.36888557246477|         1|13.967095182334086|
+-------------------+------------------+----------+------------------+
only s

In [26]:
df.limit(10).toPandas()

,date,sensor_value1,station_id,sensor_value2
0,2018-01-01 00:00:00,6.232350,1,9.291957
1,2018-01-01 00:01:00,8.681493,1,12.907931
2,2018-01-01 00:02:00,5.557534,1,8.492620
3,2018-01-01 00:03:00,0.664423,1,0.956655
4,2018-01-01 00:04:00,2.155983,1,3.329389
5,2018-01-01 00:05:00,4.772506,1,7.207562
6,2018-01-01 00:06:00,1.811211,1,2.676286
7,2018-01-01 00:07:00,9.682279,1,14.459765
8,2018-01-01 00:08:00,5.655455,1,8.410295
9,2018-01-01 00:09:00,9.368886,1,13.967095


In [32]:
df.summary().toPandas()

,summary,sensor_value1,station_id,sensor_value2
0,count,1000000,1000000,1000000
1,mean,4.999470288905015,5.5,7.398994397720761
2,stddev,2.8868357612895847,2.8722827594107754,4.347537285664498
3,min,-0.16951415673506842,1,-0.1514252213209846
4,25%,2.496727958055422,3,3.6563218221872464
5,50%,4.999603739394387,5,7.32076740277272
6,75%,7.497381284596031,8,10.979742507675812
7,max,10.16457616320874,10,17.114699127085327


In [35]:
df.select('date', 'sensor_value1').limit(10).toPandas()

,date,sensor_value1
0,2018-01-01 00:00:00,6.232350
1,2018-01-01 00:01:00,8.681493
2,2018-01-01 00:02:00,5.557534
3,2018-01-01 00:03:00,0.664423
4,2018-01-01 00:04:00,2.155983
5,2018-01-01 00:05:00,4.772506
6,2018-01-01 00:06:00,1.811211
7,2018-01-01 00:07:00,9.682279
8,2018-01-01 00:08:00,5.655455
9,2018-01-01 00:09:00,9.368886


In [57]:
df.filter((col("sensor_value1")==1) | (col("sensor_value2")==3)).show()

+----+-------------+----------+-------------+
|date|sensor_value1|station_id|sensor_value2|
+----+-------------+----------+-------------+
+----+-------------+----------+-------------+



In [38]:
df.filter(df.sensor_value1 < 0).limit(10).toPandas()

,date,sensor_value1,station_id,sensor_value2
0,2018-01-01 07:39:00,-0.018579,1,0.032505
1,2018-01-01 08:09:00,-0.048714,1,-0.009061
2,2018-01-01 12:49:00,-0.026525,1,0.074991
3,2018-01-01 18:44:00,-0.045978,1,0.006159
4,2018-01-01 20:06:00,-0.020722,1,0.096425
5,2018-01-02 02:13:00,-0.013568,1,0.081049
6,2018-01-02 02:25:00,-0.084386,1,0.082789
7,2018-01-02 12:56:00,-0.005973,1,0.032426
8,2018-01-02 19:51:00,-0.034836,1,0.093145
9,2018-01-03 09:54:00,-0.008071,1,-0.010713


In [18]:
from pyspark.sql.functions import *

In [61]:
df.filter(col('sensor_value1') < 0) \
.sort(col('sensor_value1'), ascending=False) \
.select('station_id', 'sensor_value1', 'date') \
.limit(10) \
.toPandas()

,station_id,sensor_value1,date
0,2,-0.000023,2018-02-26 10:31:00
1,2,-0.000036,2018-01-06 05:37:00
2,5,-0.000045,2018-03-02 15:38:00
3,3,-0.000054,2018-02-12 20:47:00
4,7,-0.000072,2018-02-24 17:47:00
5,6,-0.000087,2018-01-02 09:26:00
6,5,-0.000094,2018-01-18 11:59:00
7,4,-0.000095,2018-02-22 03:19:00
8,1,-0.000155,2018-02-28 13:29:00
9,7,-0.000178,2018-01-03 04:14:00


In [80]:
# EX 4.1
ex4_1 = df.filter(df.sensor_value1 < 0)

print(f"Count ({ex4_1.count()})\n\n"
      , ex4_1.limit(10).toPandas())

Count (1933)                  date  sensor_value1  station_id  sensor_value2
0 2018-01-01 07:39:00      -0.018579           1       0.032505
1 2018-01-01 08:09:00      -0.048714           1      -0.009061
2 2018-01-01 12:49:00      -0.026525           1       0.074991
3 2018-01-01 18:44:00      -0.045978           1       0.006159
4 2018-01-01 20:06:00      -0.020722           1       0.096425
5 2018-01-02 02:13:00      -0.013568           1       0.081049
6 2018-01-02 02:25:00      -0.084386           1       0.082789
7 2018-01-02 12:56:00      -0.005973           1       0.032426
8 2018-01-02 19:51:00      -0.034836           1       0.093145
9 2018-01-03 09:54:00      -0.008071           1      -0.010713


In [79]:
# EX 4.2
ex4_2 = df.filter(df.sensor_value2 < 0)

print(f"Count ({ex4_2.count()})\n\n"
      , ex4_2.limit(10).toPandas())

Count (1307)                  date  sensor_value1  station_id  sensor_value2
0 2018-01-01 08:09:00      -0.048714           1      -0.009061
1 2018-01-01 16:31:00       0.094364           1      -0.079865
2 2018-01-01 17:00:00       0.046017           1      -0.030370
3 2018-01-03 03:24:00       0.045279           1      -0.055004
4 2018-01-03 09:54:00      -0.008071           1      -0.010713
5 2018-01-03 12:04:00       0.108400           1      -0.005748
6 2018-01-03 19:36:00       0.005262           1      -0.099039
7 2018-01-03 21:24:00       0.087576           1      -0.064866
8 2018-01-04 03:37:00       0.157918           1      -0.000491
9 2018-01-06 08:26:00       0.080159           1      -0.020431


In [77]:
# EX 4.3
ex4_3 = df.filter((df.sensor_value2 < 0) & (df.sensor_value1 < 0))

print(f"Count ({ex4_3.count()})\n\n"
      , ex4_3.limit(10).toPandas())

Count (453)                  date  sensor_value1  station_id  sensor_value2
0 2018-01-01 08:09:00      -0.048714           1      -0.009061
1 2018-01-03 09:54:00      -0.008071           1      -0.010713
2 2018-01-07 11:09:00      -0.052431           1      -0.017626
3 2018-01-07 19:05:00      -0.029947           1      -0.029748
4 2018-01-08 03:38:00      -0.016698           1      -0.002929
5 2018-01-08 09:11:00      -0.046833           1      -0.015605
6 2018-01-10 03:16:00      -0.095434           1      -0.055817
7 2018-01-10 11:24:00      -0.033616           1      -0.002480
8 2018-01-10 13:57:00      -0.047937           1      -0.006325
9 2018-01-14 05:36:00      -0.034187           1      -0.044360


In [81]:
# EX 4.4
ex4_4 = df.filter(col('date') < '2018-01-07') \
.sort(col('date').desc())

print(f"Count ({ex4_4.count()})\n\n"
      , ex4_4.limit(10).toPandas())

Count (86400)
                  date  sensor_value1  station_id  sensor_value2
0 2018-01-06 23:59:00       1.616206           8       1.750975
1 2018-01-06 23:59:00       2.423315          10       3.754260
2 2018-01-06 23:59:00       3.167986           2       4.669550
3 2018-01-06 23:59:00       0.709761           5       1.060096
4 2018-01-06 23:59:00       4.467855           9       6.523746
5 2018-01-06 23:59:00       2.515432           6       3.881016
6 2018-01-06 23:59:00       2.181147           4       3.389137
7 2018-01-06 23:59:00       2.042245           3       2.965310
8 2018-01-06 23:59:00       7.174998           1      10.762004
9 2018-01-06 23:59:00       1.813730           7       2.978897


In [84]:
# EX 4.5

ex4_5 = df.filter((col('date') > '2018-01-07') & (col('date') < '2018-01-10')) \
.sort(col('date'))

print(f"Count ({ex4_5.count()})\n\n"
      , ex4_5.limit(10).toPandas())

Count (43190)

                  date  sensor_value1  station_id  sensor_value2
0 2018-01-07 00:01:00       6.892700           8       7.641647
1 2018-01-07 00:01:00       5.277072          10       7.955102
2 2018-01-07 00:01:00       3.870638           2       5.957011
3 2018-01-07 00:01:00       2.516946           5       3.905913
4 2018-01-07 00:01:00       9.302757           7      15.781736
5 2018-01-07 00:01:00       6.323936           6       9.265390
6 2018-01-07 00:01:00       2.731071           1       4.254257
7 2018-01-07 00:01:00       4.359241           4       6.623142
8 2018-01-07 00:01:00       3.927604           3       5.928673
9 2018-01-07 00:01:00       1.076772           9       1.465412


In [87]:
# EX 4.6

ex4_6 = df.filter((col('date') > '2018-01-07 11:00:00') 
                  & (col('date') < '2018-01-07 13:00:00')) \
.sort(col('date').desc())

print(f"Count ({ex4_6.count()})\n\n"
      , ex4_6.limit(10).toPandas())

Count (1190)

                  date  sensor_value1  station_id  sensor_value2
0 2018-01-07 12:59:00       7.233135           8       8.014780
1 2018-01-07 12:59:00       6.387081          10       9.561420
2 2018-01-07 12:59:00       6.821968           2      10.131801
3 2018-01-07 12:59:00       5.211718           5       7.786099
4 2018-01-07 12:59:00       8.691292           9      13.059622
5 2018-01-07 12:59:00       4.079703           6       6.139636
6 2018-01-07 12:59:00       7.783960           4      11.595297
7 2018-01-07 12:59:00       9.107154           3      13.711814
8 2018-01-07 12:59:00       7.504813           1      11.363055
9 2018-01-07 12:59:00       7.206123           7      12.031641


In [8]:
# Data for Workshop 5
!wget https://storage.googleapis.com/bd522_2021/data/sensor_3.json

--2024-03-03 02:54:35--  https://storage.googleapis.com/bd522_2021/data/sensor_3.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84182547 (80M) [application/json]
Saving to: ‘sensor_3.json’

sensor_3.json       100%[===================>]  80.28M   177MB/s    in 0.5s    

2024-03-03 02:54:36 (177 MB/s) - ‘sensor_3.json’ saved [84182547/84182547]



In [9]:
!hadoop fs -put sensor_3.json /user/dataproc/sensor_3.json
!hadoop fs -ls /user/dataproc/

Found 2 items
-rw-r--r--   2 root hadoop   62711541 2024-03-03 02:39 /user/dataproc/sensor_12.csv
-rw-r--r--   2 root hadoop   84182547 2024-03-03 02:54 /user/dataproc/sensor_3.json


In [10]:
df2 = spark.read.json("/user/dataproc/sensor_3.json")

In [93]:
df2.count()

1000000

In [95]:
df2.limit(10).toPandas()

,date,sensor_value3,station_id
0,2018-01-01 00:00:00,98.758042,001
1,2018-01-01 00:01:00,83.311504,001
2,2018-01-01 00:02:00,67.183300,001
3,2018-01-01 00:03:00,96.351818,001
4,2018-01-01 00:04:00,89.726888,001
5,2018-01-01 00:05:00,45.478606,001
6,2018-01-01 00:06:00,41.985771,001
7,2018-01-01 00:07:00,14.692583,001
8,2018-01-01 00:08:00,69.890227,001
9,2018-01-01 00:09:00,50.039344,001


In [96]:
df2.printSchema()

root
 |-- date: string (nullable = true)
 |-- sensor_value3: double (nullable = true)
 |-- station_id: string (nullable = true)



In [99]:
# Data Preparation
# Convert df2 column to correct type
df2 = df2.withColumn("station_id", col('station_id').cast('integer'))
df2 = df2.withColumn("date", col('date').cast('timestamp'))
df2.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- sensor_value3: double (nullable = true)
 |-- station_id: integer (nullable = true)



In [21]:
df2.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- sensor_value3: double (nullable = true)
 |-- station_id: string (nullable = true)



In [11]:
# 66076050 Ittichai Boonyarakthunya
df.withColumn("station_id", df.station_id)

DataFrame[date: timestamp, sensor_value1: double, station_id: int, sensor_value2: double]

In [19]:
df2 = df2.withColumn("date", to_timestamp(df2.date, "yyyy-MM-dd HH:mm:ss"))

In [20]:
df2.limit(10).toPandas()

,date,sensor_value3,station_id
0,2018-01-01 00:00:00,98.758042,001
1,2018-01-01 00:01:00,83.311504,001
2,2018-01-01 00:02:00,67.183300,001
3,2018-01-01 00:03:00,96.351818,001
4,2018-01-01 00:04:00,89.726888,001
5,2018-01-01 00:05:00,45.478606,001
6,2018-01-01 00:06:00,41.985771,001
7,2018-01-01 00:07:00,14.692583,001
8,2018-01-01 00:08:00,69.890227,001
9,2018-01-01 00:09:00,50.039344,001


In [22]:
df_123 = df.join(df2, ["station_id", "date"], "inner")

In [23]:
df_123.count()

1000000

In [24]:
df_123.sort(df_123.station_id, df_123.date).limit(20).toPandas()

,station_id,date,sensor_value1,sensor_value2,sensor_value3
0,1,2018-01-01 00:00:00,6.232350,9.291957,98.758042
1,1,2018-01-01 00:01:00,8.681493,12.907931,83.311504
2,1,2018-01-01 00:02:00,5.557534,8.492620,67.183300
3,1,2018-01-01 00:03:00,0.664423,0.956655,96.351818
4,1,2018-01-01 00:04:00,2.155983,3.329389,89.726888
5,1,2018-01-01 00:05:00,4.772506,7.207562,45.478606
6,1,2018-01-01 00:06:00,1.811211,2.676286,41.985771
7,1,2018-01-01 00:07:00,9.682279,14.459765,14.692583
8,1,2018-01-01 00:08:00,5.655455,8.410295,69.890227
9,1,2018-01-01 00:09:00,9.368886,13.967095,50.039344


In [26]:
df_123.summary().toPandas()

,summary,station_id,sensor_value1,sensor_value2,sensor_value3
0,count,1000000,1000000,1000000,1000000
1,mean,5.5,4.999470288905112,7.398994397720776,49.993474864098545
2,stddev,2.8722827594107585,2.886835761289589,4.347537285664511,28.88151439900091
3,min,1,-0.16951415673506842,-0.1514252213209846,-0.12156616126372416
4,25%,3,2.4970664977442065,3.65639694199805,24.99068482433676
5,50%,5,4.999426406861214,7.320142289988287,49.98678758476496
6,75%,8,7.4963847645179635,10.979389169779461,75.00864443053553
7,max,10,10.16457616320874,17.114699127085327,100.13130454490204


In [29]:
df_123.filter(df_123.sensor_value1 < 0) \
      .select("station_id", "date", "sensor_value1") \
      .limit(10).toPandas()

,station_id,date,sensor_value1
0,1,2018-02-14 05:31:00,-0.027290
1,3,2018-02-16 16:31:00,-0.030240
2,4,2018-03-11 07:58:00,-0.015253
3,6,2018-01-27 15:32:00,-0.008047
4,6,2018-02-25 08:00:00,-0.007614
5,7,2018-01-30 14:24:00,-0.027527
6,10,2018-02-14 17:24:00,-0.003881
7,1,2018-02-10 19:42:00,-0.075912
8,4,2018-02-07 07:35:00,-0.010811
9,4,2018-03-04 00:14:00,-0.025944


In [40]:
df_missing = df_123.withColumn("sensor_value1",
                               when(df_123.sensor_value1 < 0, None) \
                               .otherwise(df_123.sensor_value1)
                              ).withColumn("sensor_value2",
                               when(df_123.sensor_value2 < 0, None) \
                               .otherwise(df_123.sensor_value2)
                              ) .withColumn("sensor_value3",
                               when(df_123.sensor_value3 < 0, None) \
                               .otherwise(df_123.sensor_value3)
                              )

In [41]:
df_missing.summary().toPandas()

,summary,station_id,sensor_value1,sensor_value2,sensor_value3
0,count,1000000,998067,998693,999788
1,mean,5.5,5.009214936080097,7.408720672602676,50.00408265148674
2,stddev,2.872282759410742,2.8811169835414288,4.342054313699716,28.87538692422476
3,min,1,3.659785381403963E-5,3.368734710600292E-5,1.40151152320106E-5
4,25%,3,2.511536078734494,3.6702674553029344,25.016526945281722
5,50%,5,5.008910852247991,7.329484838824065,50.004711366039004
6,75%,8,7.501653185183108,10.984136624221378,75.01631411490156
7,max,10,10.16457616320874,17.114699127085327,100.13130454490204


In [42]:
df_missing.filter(df_missing.sensor_value1.isNull()).count()

1933

In [43]:
df_missing.filter(df_missing.sensor_value2.isNull()).count()

1307

In [44]:
df_missing.filter(df_missing.sensor_value3.isNull()).count()

212

In [46]:
df_all = df_missing.fillna(value=0)

In [50]:
df_all.filter(df_all.sensor_value3.isNull()).count()

0

In [53]:
df_all.groupBy("station_id").avg("sensor_value1").limit(10).toPandas()

+----------+------------------+
|station_id|avg(sensor_value1)|
+----------+------------------+
|         1| 4.983879871569178|
|         6| 5.004119994251681|
|         3| 4.999445212048462|
|         5| 5.001850919635469|
|         9| 4.999207976383456|
|         4| 5.008636293458652|
|         8| 4.996536960879175|
|         7|  4.98666113212171|
|        10| 5.011319849024109|
|         2| 5.003663026715087|
+----------+------------------+



AttributeError: 'NoneType' object has no attribute 'toPandas'

In [59]:
df_all.groupBy("station_id").avg("sensor_value1", "sensor_value2", "sensor_value3") \
.limit(10).sort("station_id").toPandas()

,station_id,avg(sensor_value1),avg(sensor_value2),avg(sensor_value3)
0,1,4.983880,7.475733,49.893118
1,2,5.003663,7.505473,50.218529
2,3,4.999445,7.499042,50.072302
3,4,5.008636,7.512784,49.953690
4,5,5.001851,7.502590,49.979242
5,6,5.004120,7.506005,49.916680
6,7,4.986661,8.477169,50.021773
7,8,4.996537,5.496359,49.835464
8,9,4.999208,7.498492,49.999791
9,10,5.011320,7.516728,50.044228
